In [135]:
# Package imports
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np
from decimal import Decimal
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from mnist import MNIST
import datetime

In [136]:
# initialise the weights for the network based on the input layers, the number of hidden layers, the number of output layers
#reference for above https://www.coursera.org/learn/deep-neural-network/lecture/RwqYe/weight-initialization-for-deep-networks
def initialise_input_weights(n_inputs, n_hidden_inputs):
 hidden_layer_weights = list()
 for i in range(n_hidden_inputs):
  weight = np.random.randn(n_inputs)*np.sqrt(1/(n_inputs)**(n_hidden_inputs-1))
  hidden_layer_weights.append(weight)
   
 input_weights = np.array([hidden_layer_weights])
 input_weights = np.reshape(input_weights, (n_inputs, n_hidden_inputs))
 return input_weights; 

def initialise_output_weights(n_hidden_inputs,n_outputs):
 output_layer_weights = list()
 for i in range(n_outputs):
  weight = np.random.randn(n_outputs)*np.sqrt(1/(n_outputs)**(n_hidden_inputs-1))
  output_layer_weights.append(weight) 
   
 output_weights = np.array([output_layer_weights])
 #if n_outputs == 1:
  #output_weights = np.array([output_layer_weights])
 #elif n_outputs > 1:
  #output_weights = np.array([output_layer_weights])  
  #output_weights = np.reshape(output_weights, (n_hidden_inputs,n_outputs))   

 return output_weights;

#initialise the bias for the network based on the number of hidden layers and the output layer bias
def initialise_bias(n_hidden_layer):
 hidden_layer_bias = list()    
 for i in range(n_hidden_layer):
  bias = np.random.random(1)[0]
  hidden_layer_bias.append(bias)
 
 output_layer_bias = [np.random.random(1)[0]]
 network_bias = [[hidden_layer_bias],[output_layer_bias]]
 return network_bias;


In [137]:
# Testing weight intialisation
input_weights = initialise_input_weights(4,2)
print("Input Weights: ",input_weights)
weights = initialise_output_weights(2,4)
print("Output Weights: ",weights)

bias = initialise_bias(1)
print("Bias:", bias)

Input Weights:  [[-0.07223614  0.68299066]
 [ 0.05021177  0.33873961]
 [ 0.37491764  0.185205  ]
 [ 0.18137765  0.05359783]]
Output Weights:  [[[-0.22632591  0.02085877  0.42755861 -0.48800597]
  [ 0.80906297  0.47078369  0.06190929  0.68706746]
  [ 0.87608869  0.00583639  0.35673269  0.99378664]
  [ 0.27084481  0.13703785  0.35662091 -0.30558057]]]
Bias: [[[0.53402582479343974]], [[0.81525647518931654]]]


In [138]:
# Sigmoid function
# g(z) = 1/ 1 + e^-z
def sigmoid(z):
 g = 1/(1 + np.exp(-z))
 return g;

# --------------- FORWARD PROPAGATION  --------------------------
# return the sigmoid_input_act - sigmoid function sum of the input layer activation 
# a_n^2 = f(W_n1^1 x1 + W_n2^1 x2 + W_n3^1 x3 +b_n^1 )
def get_input_layer_activation(input, input_weights, input_bias):
 print("Input shape ",np.shape(input))
 print("Input weight shape ",np.shape(input))
 hidden_layer = (input * input_weights)
 #print("input_layer_activation: ",input_layer_activation)
 hidden_layer = np.sum(hidden_layer, axis = 1) + input_bias * 1
 #print("Sum of Hidden Layer activation: ", hidden_layer_activation)
 hidden_layer = sigmoid(hidden_layer)
 #print("Sigmoid function of Sum of Hidden Layer activation: ", sigmoid_input_activation)
 return hidden_layer;

# get output layer activation for hidden layer
# a1_3
def get_output_layer_activation(hidden_layer, output_weights, output_bias):
 activation_output = (hidden_layer * output_weights)
 activation_output = sigmoid(np.sum(activation_output) + output_bias * 1 )
 #print("Activation Output: ",activation_output)   
 return activation_output;
 
def forward_propagation(input, input_weights, output_weights, input_bias, output_bias):
 sigmoid_input_activation = get_input_layer_activation(input, input_weights, input_bias)
 activation_output = get_output_layer_activation(sigmoid_input_activation, output_weights, output_bias)
 #print("Forward Propagation Activation Output: ",activation_output)    
 return activation_output;   

# Calculate the Total Error Sum of Squared Errors = ∑ 1/2(Y-YP)^2 
#E_total = 1/2(target_01 - out_01)^2 
# sum of squared errors of prediction
def calc_error(actual_y, target):
 error = 1/2 * np.power((actual_y - target), 2)
 return error;

# If there is more than 1 node in the output layer, sum up the calc error
def calc_total_error(actual_y, target):
 total_error = 0
 total_error += calc_error(actual_y, target)
 return total_error;


In [139]:
total_error  = calc_total_error(1, 0)
print(total_error)

0.5


In [140]:
# FUNCTIONS FOR BACK PROPAGATION OF OUTPUT
# calculate derivative of error at output layer

# Derivitive of error with reference to output
# deriv_wrt_out = -(target - output)
# change above to function
def deltaErr_wrt_out(target, output):
 result = -(target - output)
 print("Derivitive of error with reference to output:", result)
 return result;  

# calculate the derivation of the error output wrt the net
# derivout_wrt_net = output*(1-output)
# change above to function
def deltaOut_wrt_net(output):
 result = output * (1 - output)
 print("Derivitive of error output with reference to net output:", result)
 return result;   

# calculate derivative of error wrt to output layer weight OLW_Deriv
# Output Layer Weight Derivitive
def deltaErr_ow(deriv_wrt_out, derivout_wrt_net, activation):
 OLW_Deriv = deriv_wrt_out * derivout_wrt_net * activation
 print("Derivative of error with reference to output layer weight:", OLW_Deriv)
 return OLW_Deriv;
    
# FUNCTIONS FOR BACK PROPAGATION OF HIDDEN LAYER
# calculate derivative of error at hidden layer
# deriv_out_wrt_hL =  Weights2 * deriv_wrt_out *derivout_wrt_net
# print (deriv_out_wrt_hL)
# convert above to function
def deltaOut_hL(deriv_wrt_out, derivout_wrt_net, output_weights):
 deriv_out_wrt_hL = deriv_wrt_out * derivout_wrt_net * output_weights
 return deriv_out_wrt_hL;

# derivitive output in relation to net of hidden layer activation
# deriv_out_wrt_nethL = activation*(1-activation)
# convert above to function
def deltaOut_netHL(activation):
 activation = activation * (1 - activation)
 return activation;

# Derivitive in relation to input_weights
# deriv_wrt_wi = deriv_out_wrt_hL*deriv_out_wrt_nethL*Weights1
# convert above to function
def deltaErr_wi(deriv_out_wrt_hL, deriv_out_wrt_nethL, input_weights):
 deriv_wrt_wi = deriv_out_wrt_hL * deriv_out_wrt_nethL * input_weights
 return deriv_wrt_wi;

# convert the above to a function
def calc_adjusted_weights(W, deriv):
 W = W - (alpha * deriv)
 return W;

# delta error with reference to output
def delta_error_wrt_output(output, hidden_layer, target):
 deltaErr_out = deltaErr_wrt_out(target, output)
 deltaOut_net = deltaOut_wrt_net(output)
 deltaErrtot_ow = deltaErr_ow(deltaErr_out, deltaOut_net, hidden_layer)
 print("Delta Error with reference to output", deltaErrtot_ow)
 return deltaErrtot_ow, deltaErr_out, deltaOut_net;

 
# Delta Error with reference to input
def delta_error_wrt_input(hidden_layer, network_bias, deltaErr_out, deltaOut_net):
 deltaErrOut_hL = deltaOut_hL(deltaErr_out, deltaOut_net, network_bias[1])
 deltaErrOut_netHL = deltaOut_netHL(hidden_layer)
 deltaErrH_wi = deltaErr_wi(deltaErrOut_hL, deltaErrOut_netHL, network_bias[0])
 print("Delta Error with reference to input", deltaErrH_wi)
 return deltaErrH_wi;

In [141]:
 # Reading in the the small data file for the train/test datasets   
inputDataFrame = pd.read_csv("moons400.csv")  # instance variable unique to each instance
y = inputDataFrame['Class'].values   

#split up the dataset into training and test split 70/30 for training/test
train_X, test_X, train_y, test_y = train_test_split(inputDataFrame, y, test_size=0.30)

del train_X['Class']    # drop the 'Class' column from the Train and test dataframe
del test_X['Class']
print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

(nsamples, nattribs) = np.shape(train_X)
input_X = train_X
input_y = train_y


(280, 2) (280,)
(120, 2) (120,)


In [142]:
# set up weights and biases 
(train_samples, train_shape) = np.shape(inputs)
print ("train: samples =", train_samples, ", attribs =", train_shape)

# Test weights and bias initialisation based on network inputs, hidden layers, and outputs
# first line of moons.csv
#input_X =[[2.07106946, 0.41152931]]

#(nsamples, nattribs) = np.shape(input_X)
n_inputs = train_shape

n_hidden_layer = 1
n_hidden_inputs = 2
n_outputs = 1

# normalise training data
# rescale the inputs using normalization 
# mnist_train = preprocessing.normalize(filtered_images)
#print("normalised inputs",inputs)

Weights1 = np.array(initialise_input_weights(n_inputs, n_hidden_inputs), dtype=np.float64)
Weights2 = np.array(initialise_output_weights(n_hidden_inputs, n_outputs), dtype=np.float64)
#network_weights = initialise_weights(n_inputs, n_hidden_inputs, n_outputs)
bias = np.array(initialise_bias(n_hidden_layer), dtype=np.float64)

print("Length of inputs",len(inputs))
#print("actual y",input_y)
print("input Weights", len(Weights1))
print(Weights1)
print("output Weights", len(Weights2))
print("Network Bias", len(bias))

train: samples = 280 , attribs = 2
Length of inputs 280
input Weights 2
[[ 0.15610386 -0.94953597]
 [-0.57003457  0.93783941]]
output Weights 1
Network Bias 2


In [143]:
# Test weights and bias initialisation based on network inputs, hidden layers, and outputs
# Number of input nodes, number of hidden nodes, number of hidden layers and number of output nodes
n_inputs = nattribs
n_hidden_layer = 1
n_hidden_inputs = 2
n_outputs = 1

# rescale the inputs using normalization 
inputs = preprocessing.normalize(input_X)

input_weights = np.array(initialise_input_weights(n_inputs, n_hidden_inputs), dtype=np.float64)
output_weights = np.array(initialise_output_weights(n_hidden_inputs, n_outputs), dtype=np.float64)
network_bias = np.array(initialise_bias(n_hidden_layer), dtype=np.float64)


In [144]:
# Test weight setup
print(" ---- Train details ---- ")
print("Length of inputs",len(inputs))
print("Input Weights", input_weights)
print("Output Weights", output_weights)
print("Network Bias", network_bias)

 ---- Train details ---- 
Length of inputs 280
Input Weights [[ 0.25922157 -0.88883551]
 [-0.48035078 -0.78429056]]
Output Weights [[[-1.41773504]]]
Network Bias [[[ 0.72454061]]

 [[ 0.16372775]]]


In [145]:
# Original Functions
# FORWARD PROPAGATION USING FUNCTIONS
# do the above using a function
# convert calculations to function for hidden layer
def get_sigmoid_output(weights, input, bias):
    hidden_layer = input * weights 
    hidden_layer = np.sum(hidden_layer, axis=1) + bias
    hidden_layer = 1/(1+np.exp(-hidden_layer))
    return hidden_layer;

def get_output(sigmoid_out, weights, bias):
    output = (sigmoid_out*weights)
    output = 1/(1+np.exp(-(np.sum(output)+bias)))
    return output;

# get the error 
# error_o = (0.5*(target - output)**2)
# print(error_o)

#convert to a function
# I think this needs to change to accept a numpy array as well
# we should end up with a data frame we can plot against the number of executions
def get_error(t, o):
    error_o = (0.5*(t - o)**2)
    return error_o;
# FUNCTIONS FOR BACK PROPAGATION OF OUTPUT
# calculate derivative of error at output layer

# deriv_wrt_out = -(target - output)
# change above to function
def deltaErr_wrt_out(targ, outp):
    result = -(targ - outp)
    return result;  

# calculate the derivation of the error output wrt the net
# derivout_wrt_net = output*(1-output)
# change above to function
def deltaOut_wrt_net(outp):
    result = outp*(1-outp)
    return result;   

# change above to function
# calculate derivative of error wrt to output layer weight OLW_Deriv
def deltaErr_ow(deriv_wrt_out, derivout_wrt_net, activation):
    OLW_Deriv = deriv_wrt_out*derivout_wrt_net*activation
   # print(OLW_Deriv)
    return OLW_Deriv;
# FUNCTIONS FOR BACK PROPAGATION OF HIDDEN LAYER
# calculate derivative of error at hidden layer

# deriv_out_wrt_hL =  Weights2 * deriv_wrt_out *derivout_wrt_net
# print (deriv_out_wrt_hL)
# convert above to function
def deltaOut_hL(deriv_wrt_out, derivout_wrt_net, weights):
    deriv_out_wrt_hL = deriv_wrt_out *derivout_wrt_net *  weights
    return deriv_out_wrt_hL;

# deriv_out_wrt_nethL = activation*(1-activation)
# convert above to function
def deltaOut_netHL(activation):
    activation = activation*(1-activation)
    return activation;

# deriv_wrt_wi = deriv_out_wrt_hL*deriv_out_wrt_nethL*Weights1
# convert above to function
def deltaErr_wi(deriv_out_wrt_hL, deriv_out_wrt_nethL, weights):
    deriv_wrt_wi = deriv_out_wrt_hL*deriv_out_wrt_nethL*weights
    return deriv_wrt_wi;

# convert the above to a function
def calc_adjusted_weights(W, deriv):
    W = W - (alpha*deriv)
    return W;


In [146]:
alpha = 0.01
#TRAIN MODEL
print(datetime.datetime.now())
threshold=1e-5
maxrounds=3000
iter=0
error = 99.0
while abs(error) > threshold:
    for i in range(len(inputs)):
        #inputs, input_y, input_weights, output_weights,network_bias
        # START OF FORWARD PROPAGATION FUNCTION CALLS
        # test get_sigmoid_output 
        hidden_layer = get_sigmoid_output(input_weights, inputs[i], network_bias[0])
        # test get_output
        output = get_output(hidden_layer, output_weights, network_bias[1])
        # call error function
        error = get_error(input_y[i], output)
        # print("error:", error)

        # END OF FORWARD PROPAGATION FUNCTION CALLS

        # START OF BACK PROPAGATION FOR OUTPUT LAYER
        #test function deltaErr_wrt_out
        deltaErr_out = deltaErr_wrt_out(input_y[i], output)
        # print(deltaErr_out)

        #test function
        deltaOut_net = deltaOut_wrt_net(output)
        # print(deltaOut_net)

        deltaErrtot_ow = deltaErr_ow(deltaErr_out, deltaOut_net, hidden_layer)
        # print(deltaErrtot_ow)

        # END OF BACK PROPAGATION FOR OUTPUT LAYER

        # START BACK PROPAGATION OF HIDDEN LAYER
        # calculate derivative of error at hidden layer

        # test function deltaOut_hL
        deltaErrOut_hL = deltaOut_hL(deltaErr_out, deltaOut_net, output_weights)
        # print(deltaErrOut_hL)

        #test function 
        deltaErrOut_netHL = deltaOut_netHL(hidden_layer)
        # print(deltaErrOut_netHL)

        deltaErrH_wi = deltaErr_wi(deltaErrOut_hL, deltaErrOut_netHL, input_weights)
        # print(deltaErrH_wi)

        # same calculation as above but with Weights matrix 
        # calculate adjusted weights using function
        output_weights = calc_adjusted_weights(output_weights, deltaErrtot_ow)
        input_weights = calc_adjusted_weights(input_weights, deltaErrH_wi)
        #bias[1] = calc_adjusted_weights(bias[1], deltaErr_out)
       # bias[0] = calc_adjusted_weights(bias[0], 0.5*np.sum(deltaErrOut_hL, dtype=np.float64))

        # END OF BACK PROPAGATION OF HIDDEN LAYER

    iter=iter+1
    
    if (iter > maxrounds):
        break

print ("\nFinished after ", iter, " error =", error, "target=", input_y[i], ", output=", output, "input final=", inputs[i])    
print(datetime.datetime.now())
print ("Weights1 adjusted:", input_weights)
print ("Weights2 adjusted:", output_weights)

2018-02-21 14:32:00.282103

Finished after  3001  error = [[ 0.12380719]] target= 1 , output= [[ 0.50239134]] input final= [ 0.9160975  -0.40095557]
2018-02-21 14:32:36.562047
Weights1 adjusted: [[[ 0.00624119 -0.00336873]
  [-0.01156525 -0.0029725 ]]]
Weights2 adjusted: [[[-4.92913402  4.69019771]]]


In [147]:
print(np.shape(inputs))
print(np.shape(input_y))
print(np.shape(input_weights))
print(np.shape(output_weights))
print(np.shape(network_bias))

(280, 2)
(280,)
(1, 2, 2)
(1, 1, 2)
(2, 1, 1)


In [148]:
# reference https://pypi.python.org/pypi/python-mnist
# https://github.com/sorki/python-mnist/blob/master/mnist/loader.py
# TRAIN MNIST DATA

mndata = MNIST('./mnist')
images, labels = mndata.load_training()

processed_images = mndata.process_images_to_numpy(images)
# print(processed_images[0:2])

target = np.array(labels)
# print(target[0:2])

In [149]:
# set up 2 lists to filter the MNIST data into
# keeping only 0 and 6 to classify
# TRAIN MNIST DATA

filtered_labels = []
filtered_images = []
for i in range(len(target)):
    if target[i]==0 or target[i]==6:
        filtered_labels.append(target[i])
        filtered_images.append(processed_images[i])
        
# remap the value 6 to 1, so classification is binary

#print(filtered_labels[0:10])

for i in range(len(filtered_labels)):
    if filtered_labels[i]==6:
        filtered_labels[i]=1

#print(filtered_labels[0:10])

In [150]:
# convert the lists to arrays
# TRAIN MNIST DATA
filtered_labels = np.array(filtered_labels)

filtered_images = np.array(filtered_images)

In [151]:
# MNIST TEST DATA
test_images, test_labels = mndata.load_testing()

test_images = mndata.process_images_to_numpy(test_images)
#print(processed_images[0:2])

test_target = np.array(test_labels)

# set up 2 lists to filter the MNIST data into
# keeping only 0 and 6 to classify
# TEST MNIST DATA

filtered_test_labels = []
filtered_test_images = []
for i in range(len(target)):
    if target[i]==0 or target[i]==6:
        filtered_test_labels.append(target[i])
        filtered_test_images.append(processed_images[i])
        
# remap the value 6 to 1, so classification is binary

#print(filtered_labels[0:10])

for i in range(len(filtered_test_labels)):
    if filtered_test_labels[i]==6:
        filtered_test_labels[i]=1
        
# convert the lists to arrays
# TEST MNIST DATA
filtered_test_labels = np.array(filtered_test_labels)

filtered_test_images = np.array(filtered_test_images)        

In [152]:
print(filtered_test_labels[0:10])


[0 1 1 0 1 0 1 0 1 0]


In [153]:
# set up weights and biases for MNIST data, get nattribs value
# TRAIN MNIST DATA

(train_samples, train_shape) = np.shape(filtered_images)
print ("train: samples =", train_samples, ", attribs =", train_shape)

# Test weights and bias initialisation based on network inputs, hidden layers, and outputs
# first line of moons.csv
#input_X =[[2.07106946, 0.41152931]]

#(nsamples, nattribs) = np.shape(input_X)
n_inputs = train_shape
n_hidden_layer = 1
n_hidden_inputs = 2
n_outputs = 1

# normalise training data
# rescale the inputs using normalization 
mnist_train = preprocessing.normalize(filtered_images)
#print("normalised inputs",inputs)

input_weights = np.array(initialise_input_weights(n_inputs, n_hidden_inputs))
output_weights = np.array(initialise_output_weights(n_hidden_inputs, n_outputs))
#network_weights = initialise_weights(n_inputs, n_hidden_inputs, n_outputs)
network_bias = np.array(initialise_bias(n_hidden_layer))

print("Length of inputs",len(filtered_images))
#print("actual y",input_y)
print("input Weights", len(input_weights))
print("output Weights", len(output_weights))
print("Network Bias", len(network_bias))

train: samples = 11841 , attribs = 784
Length of inputs 11841
input Weights 784
output Weights 1
Network Bias 2


In [154]:
## TEST MODEL
inputs = preprocessing.normalize(test_X)
target = test_y
target = np.array(target)
print(len(inputs))
print(len(target))
#tol=1e-7
#maxrounds=1000
#iter=0
error = 99.0
#while abs(error) > tol:
error_res = []
result = []
#result = np.array(result)

for i in range(len(inputs)):
    print ("Index: ",i, " Input: ", inputs[i])
    print ("Index: ",i," Target value: ", target[i])
        
    ### START OF FORWARD PROPAGATION FUNCTION CALLS
    # test get_sigmoid_output 
    hidden_layer = get_sigmoid_output(input_weights, inputs[i], network_bias[0])
    ##print ("hidden_layer result:", hidden_layer)
        
    # test get_output
    output = get_output(hidden_layer, output_weights, network_bias[1])
    #  print ("output:", output)
    result.append(output)
        
    # call error function
    error = get_error(target[i], output)
    error_res.append(error)
    #print("error:", error)
        
    # END OF FORWARD PROPAGATION FUNCTION CALLS
result = np.array(result)  
error_res = np.array(error_res)
target = np.array(target)
#print("Target:", target)        
#print("Results: ", result)
#print("Error:", error_res)

120
120
Index:  0  Input:  [ 0.83369257  0.55222885]
Index:  0  Target value:  0
Index:  1  Input:  [-0.54077318 -0.84116846]
Index:  1  Target value:  1
Index:  2  Input:  [ 0.99991817 -0.0127925 ]
Index:  2  Target value:  1
Index:  3  Input:  [ 0.13858785  0.99035014]
Index:  3  Target value:  1
Index:  4  Input:  [ 0.75767119 -0.65263648]
Index:  4  Target value:  1
Index:  5  Input:  [ 0.95154886  0.3074976 ]
Index:  5  Target value:  1
Index:  6  Input:  [-0.99655695  0.08291103]
Index:  6  Target value:  0
Index:  7  Input:  [ 0.99452772 -0.10447307]
Index:  7  Target value:  1
Index:  8  Input:  [ 0.99969382  0.02474393]
Index:  8  Target value:  1
Index:  9  Input:  [ 0.89158845 -0.45284659]
Index:  9  Target value:  1
Index:  10  Input:  [-0.90891162  0.41698882]
Index:  10  Target value:  0
Index:  11  Input:  [ 0.99936025  0.03576434]
Index:  11  Target value:  1
Index:  12  Input:  [ 0.05859329  0.99828194]
Index:  12  Target value:  0
Index:  13  Input:  [ 0.97561738 -0.2

Index:  115  Target value:  1
Index:  116  Input:  [ 0.95242449  0.30477464]
Index:  116  Target value:  0
Index:  117  Input:  [ 0.96201311 -0.27300325]
Index:  117  Target value:  1
Index:  118  Input:  [ 0.99508722  0.09900215]
Index:  118  Target value:  1
Index:  119  Input:  [-0.17222527  0.98505759]
Index:  119  Target value:  0


In [155]:
# calculate the accuracy of the NN
tol=1e-7
print(len(target))
print(len(result))
print(tol)
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(target)):
        
        if (target[i] == 1 and target[i]-result[i] <= tol):
            TP = TP + 1
            print("True Positive: ", target[i], " - ", result[i])

        if (target[i] == 0 and target[i]-result[i] <= tol):
            TN = TN + 1
            print("True Negative: ",target[i], " - ", result[i])
            
        if (target[i] == 1 and target[i]-result[i] > tol):
            FP = FP + 1
            print("False Positive: ",target[i], " - ", result[i])
            
        if (target[i] == 0 and target[i]-result[i] > tol):
            FN = FN + 1
            print("False Negative: ",target[i], " - ", result[i])
        
print("True Positive: ", TP)
print("True Negative: ", TN)
print("False Positive: ", FP)
print("False Negative: ", FN)


120
120
1e-07
True Negative:  0  -  [[  2.87856318e-99]]
False Positive:  1  -  [[  2.76486326e-99]]
False Positive:  1  -  [[  2.86357620e-99]]
False Positive:  1  -  [[  2.86407842e-99]]
False Positive:  1  -  [[  2.82778111e-99]]
False Positive:  1  -  [[  2.87415724e-99]]
True Negative:  0  -  [[  2.77930508e-99]]
False Positive:  1  -  [[  2.85969545e-99]]
False Positive:  1  -  [[  2.86505872e-99]]
False Positive:  1  -  [[  2.84141278e-99]]
True Negative:  0  -  [[  2.79547977e-99]]
False Positive:  1  -  [[  2.86548219e-99]]
True Negative:  0  -  [[  2.86073621e-99]]
False Positive:  1  -  [[  2.85430143e-99]]
True Negative:  0  -  [[  2.82453612e-99]]
True Negative:  0  -  [[  2.87907008e-99]]
False Positive:  1  -  [[  2.85633838e-99]]
False Positive:  1  -  [[  2.82526763e-99]]
True Negative:  0  -  [[  2.86566240e-99]]
True Negative:  0  -  [[  2.83068644e-99]]
False Positive:  1  -  [[  2.83338605e-99]]
False Positive:  1  -  [[  2.84341376e-99]]
False Positive:  1  -  [[ 